This notebook showcases Refinitiv's Screening and Sector Aggregation capabilities by leveraging our forward estimates data (NTM) on public companies that have high price momentum. The screening parameters we chose are the following:<br>

 -Universe = Public Companies<br>
 -Market Cap = Between 1 bln to 5 bln USD<br>
 -Country of Headquarters = US<br>
 -Instrument Type = Ordinary Shares<br>
 -Total 52 Week Return >= 5%

In [ ]:
import refinitiv.data as rd
import numpy as np
import pandas as pd
rd.open_session()

In [1]:

# Screening the public equity universe with our chosen parameters
screen = 'SCREEN(U(IN(Equity(active,public,primary))), BETWEEN(TR.CompanyMarketCap(Scale=6),1000.00,5000.00), IN(TR.HQCountryCode,US), IN(TR.InstrumentType, "Ordinary Shares"), TR.TotalReturn52Wk>=5, CURN=USD)'

# Adding estimates to the screened universe - Fwd P/E, Fwd EV/EBITDA and Fwd P/BVPS
companies = rd.get_data(universe=[screen], fields=['TR.CommonName', 'TR.CompanyMarketCap(Scale=6)',
                        'TR.TotalReturn52Wk', 'TR.GICSSector', 'TR.FwdPE', 'TR.FwdEVToEBITDA', 'TR.FwdPricetoBVPerShare'])

# Rename Column
companies.rename(
    columns={'Company Market Cap': 'Company Market Cap (MM)'}, inplace=True)

# Removing rows without an estimate value
companies['Forward P/E (Daily Time Series Ratio)'].replace('',
                                                           np.nan, inplace=True)
companies['Forward Enterprise Value To EBITDA (Daily Time Series Ratio)'].replace(
    '', np.nan, inplace=True)
companies['Forward Price To Book Value Per Share (Daily Time Series Ratio)'].replace(
    '', np.nan, inplace=True)
companies.dropna(subset=['Forward Price To Book Value Per Share (Daily Time Series Ratio)',
                 'Forward P/E (Daily Time Series Ratio)', 'Forward Enterprise Value To EBITDA (Daily Time Series Ratio)'], inplace=True)

# Averaging and aggregating across GICS Sectors
means = companies.groupby('GICS Sector Name').mean().round(2)
means

/var/folders/hd/5zrct37502z2kq6ml9_65cn80000gn/T/ipykernel_34621/1948108082.py:27: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  means = companies.groupby('GICS Sector Name').mean().round(2)


,Company Market Cap (MM),52 Week Total Return,Forward P/E (Daily Time Series Ratio),Forward Enterprise Value To EBITDA (Daily Time Series Ratio),Forward Price To Book Value Per Share (Daily Time Series Ratio)
GICS Sector Name,,,,,
Communication Services,2580.82,21.68,48.81,20.49,-7.2
Consumer Discretionary,2507.79,23.42,17.86,9.93,0.7
Consumer Staples,2878.92,24.11,22.58,13.93,1.14
Energy,2459.28,61.41,11.71,5.5,2.43
Financials,2853.44,23.26,10.62,135.66,3.08
Health Care,3253.9,32.97,322.1,14.16,3.56
Industrials,2944.82,26.63,20.26,11.18,4.14
Information Technology,3070.26,28.1,143.43,24.59,4.72
Materials,2689.15,36.41,12.92,7.44,2.33


In [ ]:
rd.close_session()